**Task 1.** Необходимо написать скрипт, который выполняет мониторинг средних продаж в реальном времени. Каждую минуту вы получаете данные о суммах продаж и должны рассчитать среднее значение продаж нарастающим итогом, чтобы иметь представление о текущих показателях.

In [3]:
def calc_avg(numbers):
    if not numbers:
        return None

    return sum(numbers)/len(numbers)

sales = []
while True:
    amount = float(input('amount>>'))
    sales.append(amount)
    result = calc_avg(sales)
    print(f'AVG: {result}')

    ans = input('Continue (Y/N)').upper()
    if ans == 'N':
        break

amount>> 1


AVG: 1.0


Continue (Y/N) 2
amount>> 2


AVG: 1.5


Continue (Y/N) N


In [6]:
def calc_avg_gen():
    total = 0
    count = 0
    while True:
        number = yield
        if number:
            total += number
            count += 1
            yield total / count


sales = calc_avg_gen()
next(sales)

while True:
    amount = float(input('amount>>'))
    avg = sales.send(amount)
    
    print(f'AVG: {avg}')

    ans = input('Continue (Y/N)').upper()
    if ans == 'N':
        break
    next(sales)

amount>> 1


AVG: 1.0


Continue (Y/N) Y
amount>> 2


AVG: 1.5


Continue (Y/N) Y
amount>> 3


AVG: 2.0


Continue (Y/N) N


In [ ]:
def calc_avg_gen():
    total = 0
    count = 0
    while True:
        number = yield 0 if count == 0 else total / count
        if number:
            total += number
            count += 1


sales = calc_avg_gen()
next(sales)

while True:
    amount = float(input('amount>>'))
    avg = sales.send(amount)

    print(f'AVG: {avg}')

    ans = input('Continue (Y/N)').upper()
    if ans == 'N':
        break

**Task 2.**Создать скрипт, задачей которого является мониторинг транзакций в реальном времени и выявление подозрительных операций. Банк установил порог, превышение которого указывает на потенциально мошеннические транзакции. Этот порог можно динамически менять в зависимости от ситуации.


In [10]:
import json
from datetime import date

def monitor(default_limit=10_000):
    anomalies = []
    limit = default_limit

    try:
        while True:
            item = yield
    
            if isinstance(item, dict) and 'id' in item and 'amount' in item:
                if item['amount'] > limit:
                    anomalies.append(item)
                else:
                    print(f'Transaction {item["id"]} is successfull')

            elif isinstance(item, int):
                limit = item
            elif isinstance(item, str) and item.lower() == 'get_anomalies':
                yield anomalies
            else:
                raise ValueError()
    finally:
        if anomalies:
            with open(f'{date.today()}.json', 'w') as f:
                json.dump(anomalies, f, indent=4)

    


monitoring = monitor()
next(monitoring)

monitoring.send({"id": 1, "amount": 500})
monitoring.send({"id": 2, "amount": 15000})
monitoring.send({"id": 3, "amount": 5000})
monitoring.send({"id": 4, "amount": 25000})


monitoring.close()

Transaction 1 is successfull
Transaction 3 is successfull


In [ ]:
def gen_func():
    i = 0
    while True:
        try:

            yield i
            i += 1
        except ValueError:
            i = 0


gen_obj = gen_func()

for item in gen_obj:
    print(item)
    if item >= 10_000:
        gen_obj.throw(ValueError)

In [18]:
import json
import os
from datetime import datetime


def parse_date(date_str):
    try:
        return datetime.strptime(date_str, "%d-%m-%Y").date()
    except ValueError:
        raise ValueError(f"Некорректный формат даты: {date_str}. Ожидается формат DD-MM-YYYY.")


def validate_iban(iban):
    return len(iban) < 15 or not iban[:2].isalpha() or not iban[2:].isdigit()


def read_users_from_json(file_name):
    if not os.path.exists(file_name):
        raise FileNotFoundError(f'{file_name} not found')

    with open(file_name) as f:
        data = json.load(f)

    users = []
    for user in data:
        try:
            first_name = user["firstName"]
            last_name = user["lastName"]
            date_of_birth = parse_date(user["dateOfBirth"])
            iban = user["iban"]

            if validate_iban(iban):
                raise ValueError(f'Error in IBAN for user {first_name} {last_name}')

            users.append(
                {
                    'user': first_name + last_name,
                    'dateOfBirth': date_of_birth,
                    'iban': iban,
                }
            )
        except KeyError as e:
            print('Error in dict')
        except ValueError as e:
            print(e)
    return users


for item in read_users_from_json('data.json'):
    print(item)

[{'firstName': 'Ivan', 'lastName': 'Petrov', 'dateOfBirth': '15-03-2017', 'iban': 'DE1234 0000 0000 01011 333'}, {'firstName': 'Maria', 'lastName': 'Ivanova', 'dateOfBirth': '10-12-1990', 'iban': 'FR7634 1234 5678 9101 2345'}, {'firstName': 'Alex', 'lastName': 'Smith', 'dateOfBirth': '12-06-1985', 'iban': 'INVALID_IBAN'}]
